# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [83]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy
from sklearn.decomposition import TruncatedSVD
import joblib
import warnings
warnings.filterwarnings("ignore")
credits=pd.read_csv('/kaggle/input/movie-dataset/credits.csv')
keywords=pd.read_csv('/kaggle/input/movie-dataset/keywords.csv')
links=pd.read_csv('/kaggle/input/movie-dataset/links.csv')
links_small=pd.read_csv('/kaggle/input/movie-dataset/links_small.csv')
movies_metadata=pd.read_csv('/kaggle/input/movie-dataset/movies_metadata.csv')
ratings_small=pd.read_csv('/kaggle/input/movie-dataset/ratings_small.csv')

In [6]:
import sys
arr=[-2, -3, 4, -1, -2, 1, 5, -3]
max_ending_so_far=-sys.maxsize-1
max_ending_here=0
for i in arr:
    max_ending_here=max_ending_here+i
    if(max_ending_so_far<max_ending_here):
        max_ending_so_far=max_ending_here
    if(max_ending_here<0):
        max_ending_here=0
print(max_ending_so_far)


7


In [84]:
credits

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


In [85]:
keywords.head(5)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [86]:
links.head(5)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [87]:
links_small.head(5)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [88]:
movies_metadata.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [89]:
import ast
def genres(text):
    genre=''
    text=ast.literal_eval(text)
    for i in text:
        genre+=i['name']+' '
    return genre[:-1]

In [90]:
def production_companies_names(text):
    company=''
    text=ast.literal_eval(text)
    for i in text:
        company+=i['name'].replace(" ",'')+" "
    return company[:-1]

In [91]:
def production_countries_names(text):
    country=''
    text=ast.literal_eval(text)
    for i in text:
        country+=i['name'].replace(" ",'')+" "
    return country[:-1]

In [92]:
def preprocess(movies,ratings):
    movies.drop(columns=['belongs_to_collection','homepage','tagline'],inplace=True)
    movies['inference_title']=movies['title']
    total_user_ratings=ratings.groupby(by=['userId']).count()['rating'].reset_index()
    active_user_ratings=ratings[ratings['userId'].isin(total_user_ratings[total_user_ratings['rating']>50]['userId'])]
    movies['genres_name']=movies['genres'].apply(genres)
    movies=movies[movies['genres_name']!='']
    movies=movies[movies['vote_count']>movies['vote_count'].quantile(0.8)]
#     genres_id_dict={}
#     for text in movies['genres'].values:
#         text=ast.literal_eval(text)
#         for i in text:
#             genres_id_dict[i['id']]=i['name']
    movies.drop(columns=['original_language','original_title','release_date','status','genres','video'],inplace=True)
    movies.dropna(inplace=True)
    movies['production_companies_names']=movies['production_companies'].apply(production_companies_names)
    movies['production_countries_names']=movies['production_countries'].apply(production_countries_names)
    movies.drop(columns=['production_companies','production_countries'],inplace=True)
    movies['adult']=movies['adult'].str.lower()
    movies['overview']=movies['overview'].str.lower()
    movies['title']=movies['title'].str.lower()
    movies['genres_name']=movies['genres_name'].str.lower()
    movies['production_companies_names']=movies['production_companies_names'].str.lower()
    movies['production_countries_names']=movies['production_countries_names'].str.lower()
    movies['adult']=movies['adult'].apply(lambda x: 'notadult' if x=='false' else 'adult')
    movies['training_text'] = movies['genres_name'] + ' ' + movies['title'] + ' ' + movies['overview'] + ' ' + movies['adult'] +' ' + movies['production_companies_names'] + ' ' + movies['production_countries_names']
    movies.drop(columns=['adult','overview','spoken_languages','production_companies_names','production_countries_names'],inplace=True)
    movies['title']=movies['title'].str.replace(" ",'')
    return movies,ratings

In [93]:
movies,ratings=preprocess(movies_metadata,ratings_small)

In [94]:
def content_model_train(movies,path):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    bert_model = BertModel.from_pretrained('bert-base-uncased')

    movie_embeddings = []

    for movie_description in movies['training_text']:
        tokens = tokenizer.encode(movie_description, truncation=True, padding=True, return_tensors='pt')
        with torch.no_grad():
            outputs = bert_model(tokens)
            embeddings = outputs['last_hidden_state'][:, 0, :]
            movie_embeddings.append(embeddings)

    movie_embeddings = torch.cat(movie_embeddings, dim=0)

    similarity_matrix = cosine_similarity(movie_embeddings, movie_embeddings)
    model_directory=path
    tokenizer.save_pretrained(model_directory)
    bert_model.save_pretrained(model_directory)
    np.save(f'{path}similarity_matrix.npy', similarity_matrix)
    return similarity_matrix

In [95]:
# similarity_matrix=model_train(movies,'/kaggle/working/')

In [96]:
def content_inference(movie_name,movies,similarity_matrix=None):
    if(similarity_matrix==None):
        similarity_matrix=np.load('/kaggle/working/similarity_matrix.npy')
    movie_name=movie_name.lower().replace(" ",'')
    input_movie_index = (movies[movies['title']==movie_name]['id']).index[0]
    similar_movies_indices = similarity_matrix[input_movie_index].argsort()[::-1][1:6]
    similar_movies = movies.iloc[similar_movies_indices]['title']
    print(f"Top 5 Movies Similar to '{movies_metadata.iloc[input_movie_index]['title']}':")
    print(similar_movies)

In [97]:
content_inference('heat',movies)

Top 5 Movies Similar to 'Heat':
13948                 orphan
4996               dragonfly
13693    terminatorsalvation
9828                guesswho
18237               flypaper
Name: title, dtype: object


In [98]:
def collaborative_model_train(active_user_ratings):
    reader = Reader(line_format='user item rating timestamp', sep='\t', rating_scale=(0.5, 5), skip_lines=1)
    data = Dataset.load_from_df(active_user_ratings[['userId', 'movieId', 'rating']], reader)
    trainset, testset = train_test_split(data, test_size=0.2, random_state=42)
    model = SVD(n_factors=20, random_state=42)
    model.fit(trainset)
    predictions = model.test(testset)
    svd_model = TruncatedSVD(n_components=100)  # Adjust the parameters based on your actual model
    joblib.dump(svd_model, 'svd_model.joblib')
    loaded_svd_model = joblib.load('svd_model.joblib')
#     print("Explained variance ratio:", sum(svd_model.explained_variance_ratio_))
    print(accuracy.rmse(predictions))
    return model

In [99]:
svd_model=collaborative_model_train(ratings)

RMSE: 0.8992
0.8992173663695584


In [100]:
def collaborative_inference(svd_model,user_id,ratings):
#     loaded_svd_model = joblib.load('/kaggle/working/svd_model.joblib')
    all_movie_ids = ratings['movieId'].unique()

    user_movie_combinations = [(user_id, movie_id, 0) for movie_id in all_movie_ids]

    predictions = svd_model.test(user_movie_combinations)

    recommendations = [(pred.iid, pred.est) for pred in predictions]

    recommendations.sort(key=lambda x: x[1], reverse=True)

    top_n_recommendations = recommendations[:10]
    print("Top Recommendations for User {}: {}".format(user_id, top_n_recommendations))

In [101]:
collaborative_inference(svd_model,20,ratings)

Top Recommendations for User 20: [(6016, 4.043046785805066), (1172, 3.9822702975659814), (1217, 3.9711546802364515), (2542, 3.961258356359624), (3462, 3.943831122251715), (1945, 3.8973316670302385), (1228, 3.89598051166504), (1192, 3.895721110173201), (2064, 3.8917534927166564), (745, 3.870570216231786)]


### hybrid

In [102]:
def hybrid(svd_model,movies,ratings,user_id,movie_name,loaded_similarity_matrix=None):
    if(loaded_similarity_matrix==None):
        loaded_similarity_matrix = np.load('/kaggle/working/similarity_matrix.npy')
    movies.reset_index(inplace=True)
    movies.drop(columns='index',inplace=True)
    movie_name=movie_name.lower().replace(" ",'')
    input_movie_index = (movies[movies['title']==movie_name]['id']).index[0]
    loaded_similarity_matrix[input_movie_index].argsort()[::-1]
    recommended_movies=movies.iloc[loaded_similarity_matrix[input_movie_index].argsort()[::-1][1:]]
    all_movie_ids = ratings['movieId'].unique()

    user_movie_combinations = [(user_id, movie_id, 0) for movie_id in all_movie_ids]
    predictions = svd_model.test(user_movie_combinations)

    recommendations = [(pred.iid, pred.est) for pred in predictions]

    recommendations.sort(key=lambda x: x[1], reverse=True)
    recommendations_list=[ele[0] for ele in recommendations]
    recommended_movies=recommended_movies.set_index('id')
    recommendations_list = [str(idx).strip() for idx in recommendations_list]
    recommended_movies=recommended_movies[recommended_movies.index.isin(np.array(recommendations_list))]
    return recommended_movies

In [103]:
user_id=20
movie='heat'
hybrid(svd_model,movies,ratings,user_id,movie)['inference_title']

,budget,imdb_id,popularity,poster_path,revenue,runtime,title,vote_average,vote_count,inference_title,genres_name,training_text
id,,,,,,,,,,,,
941,15000000,tt0093409,13.183804,/drp80fBSWhz0vAmzUibXDyn0Pjo.jpg,120207127.0,110.0,lethalweapon,7.0,1367.0,Lethal Weapon,adventure action comedy thriller crime,adventure action comedy thriller crime lethal ...
943,35000000,tt0104714,10.786007,/pr5RcT2TdJ09PkETdhRiwixf8PW.jpg,321731527.0,118.0,lethalweapon3,6.4,824.0,Lethal Weapon 3,adventure action comedy thriller crime,adventure action comedy thriller crime lethal ...
8984,55000000,tt0109635,11.812244,/qbwcjmHnXkeE8l7wNBsdnlrp3J4.jpg,214015089.0,123.0,disclosure,5.9,217.0,Disclosure,drama thriller crime mystery romance,drama thriller crime mystery romance disclosur...
6723,10000000,tt0100514,8.110591,/eKNeRE7uxShUZavtuAJ5YY2ViYH.jpg,21633874.0,120.0,therookie,5.6,134.0,The Rookie,action crime drama thriller,action crime drama thriller the rookie veteran...
521,1400000,tt0046912,7.853231,/xrpK1PyckNWmRxU4kZURfaCyboS.jpg,3000000.0,104.0,dialmformurder,7.9,539.0,Dial M for Murder,crime mystery thriller,crime mystery thriller dial m for murder an ex...
...,...,...,...,...,...,...,...,...,...,...,...,...
919,0,tt0275230,2.925291,/3ySU3xS4g9KeJNCNDbVniOs3KmG.jpg,0.0,48.0,blood:thelastvampire,6.6,76.0,Blood: The Last Vampire,fantasy animation horror comedy thriller scien...,fantasy animation horror comedy thriller scien...
2750,0,tt0274309,12.655728,/2DJ3tP4yKYFodXXqVeLJVjE2Ssw.jpg,0.0,117.0,24hourpartypeople,6.8,109.0,24 Hour Party People,comedy drama,"comedy drama 24 hour party people in 1976, ton..."
439,0,tt0053779,11.063822,/mZJ25Th65B2KXM57uIiEzO3Aw81.jpg,19516000.0,174.0,ladolcevita,7.9,305.0,La Dolce Vita,comedy drama,comedy drama la dolce vita episodic journey of...
